In [1]:
import gzip
import pandas as pd
import os

In [2]:
innerBC_list = pd.read_csv('/syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/SPATIAL/T2/fastq/sample.list',header=None)

In [3]:
innerBC_list.iloc[:,0].to_list()

['AACGTG',
 'AACTGC',
 'AAGCTC',
 'AAGTCG',
 'AATCCG',
 'AATCGC',
 'ACAGTG',
 'ACGATC',
 'ACTCTG',
 'ACTTGA',
 'ATAGCG',
 'ATCACG',
 'ATCCAG',
 'ATGCCA',
 'ATGTTC',
 'ATTCCG',
 'staticBC']

In [4]:
barcodeToPos_df = pd.read_csv('/syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/SPATIAL/T2/barcodeToPos.txt',sep='\t',header=None)
barcodeToPos_df.columns = ['CID','x','y']
barcodeToPos_df['x_y'] = barcodeToPos_df['x'].astype('str') + '_' + barcodeToPos_df['y'].astype('str')

In [5]:
barcodeToPos_df

,CID,x,y,x_y
0,ACGACCCTGACTTGTCATAGACGCC,11758,14699,11758_14699
1,AATGTATGAATACGGATGTCTGTTG,11757,14699,11757_14699
2,AAACCTCTACCATACATGCCCAGGT,11756,14699,11756_14699
3,CCAAGGCGAGAAGAGTTATCATCAG,11753,14699,11753_14699
4,GGGGTCAAGCCTATTTATCGTTGAC,11751,14699,11751_14699
...,...,...,...,...
158077859,GAGGTTCGCATGGGGCCAGTGACAA,1327,10067,1327_10067
158077860,TTCGGGATAGCTGATTCGGATAAAC,7789,9421,7789_9421
158077861,CAGCCCATAGCGATTTCTGGCCGCT,5249,8466,5249_8466
158077862,AACCCATTCCGTCAGATTTCACGAC,7790,9421,7790_9421


In [6]:
barcodeToPos_dict = barcodeToPos_df.set_index('x_y')['CID'].to_dict()

In [7]:
innerBC_reads_count = {}
innerBC_reads_count['r1'] = {}
fastq_path = '/syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/SPATIAL/T2/fastq' + '/{}_{}.fq.gz'

for innerBC in innerBC_list.iloc[:,0].to_list():
    print(innerBC)
    R1 = gzip.open(fastq_path.format(innerBC+'_mapped','r1'))
    while True:
        r1_line = R1.readline().decode('utf-8')
        if r1_line == '' : break
        if r1_line[0] == '@': 
            read1_name = r1_line
            CID_pos = read1_name.split('CB:Z:')[1].split('|||')[0]
            CID = barcodeToPos_dict[CID_pos]
            read1_seq = R1.readline().decode('utf-8')
            read1_seq_corrected = read1_seq[0:36] + CID + read1_seq[61:151]
            read1_QI = R1.readline().decode('utf-8')
            read1_baseQ = R1.readline().decode('utf-8')
            read1 = [read1_name , read1_seq_corrected , read1_QI , read1_baseQ]
            if innerBC in innerBC_reads_count['r1']:
                innerBC_reads_count['r1'][innerBC] += read1
            else:
                innerBC_reads_count['r1'][innerBC] = read1


staticBC


In [8]:
for fq,pooled_reads in innerBC_reads_count.items():
    for innerBC,reads_set in pooled_reads.items():
        with open('/syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/SPATIAL/T2/fastq'+'/'+innerBC+'_mapped_corrected'+'_'+fq+'.fq','w+') as f:
            f.writelines(reads_set)

In [16]:
os.system('ls /syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/SPATIAL/T2/fastq/*_corrected_r1.fq|xargs -I {} gzip {}')

0

In [17]:
print('ok')

ok
